In [ ]:
import sys
import os
os.environ['OPP4SAR_DIR'] = '/home/jhewers/Documents/meng_project/code/'
sys.path.insert(0,os.getenv('OPP4SAR_DIR'))
DIR = os.path.join(os.getenv('OPP4SAR_DIR'),'generated_data/prob_map_3')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({
    'axes.grid':False,
    'figure.dpi':300,
    'image.cmap':'gray',
    'legend.loc':'upper left',
    'legend.fancybox':True,
    })
def legend_decorator(method):
    def decorate_legend(*args,**kwargs):
        cust_kwargs = {'bbox_to_anchor':(1.05, 1)}
        cust_kwargs.update(kwargs)
        return method(*args,**cust_kwargs)
    return decorate_legend
plt.legend = legend_decorator(plt.legend)

import json
import numpy as np
import scipy.stats as st
import scipy as sp
from PIL import ImageColor
import re
import os
import csv

from src.json_helpers import GlobalJsonDecoder
from src.data_models.positional.waypoint import Waypoint, Waypoints
from src.data_models.probability_map import ProbabilityMap
from src.waypoint_generation import WaypointFactory, WaypointAlgSettings
from src.enums import WaypointAlgorithmEnum
from src.simulation.simulation import SimRunnerOutput
from src.waypoint_generation.waypoint_settings import SarGenOutput, WpGenOutput

In [ ]:
from src.waypoint_generation import CostFunc
cost_func = CostFunc()

wp_gen_settings = WaypointAlgSettings.Global()

In [ ]:
reg = r'output_(sim|wp|sar)_([a-zA-Z0-9]{1,8})\.json'
data_sar = {}
data_sim = {}
data_wp = {}

R=15
k = None
for f in [file_ for file_ in os.listdir(DIR) if not os.path.isdir(file_)]:
    re_search = re.search(reg,f)
    if re_search is None:
        continue
    type_ = re_search.groups()[0]
    key = re_search.groups()[1]
    if k is None: k = key

    with open(os.path.join(DIR,f),'r') as file_:
        data = json.load(file_,cls=GlobalJsonDecoder)
        print(f"{f} | {type(data)}")            
        if isinstance(data,SarGenOutput):
            data_sar[key] = data
        elif isinstance(data,WpGenOutput):
            data_wp[key] = data
        elif isinstance(data,SimRunnerOutput):
            data_sim[key] = data

In [ ]:
# PLOT WAYPOINTS

def plot(alg,d,img,pre=""):
    fig, ax = plt.subplots()
    
    ax.imshow(img)
    wps = d.data[str(alg)]['wps']
    
    ax.plot(wps.x,wps.y,'r',linewidth=3)
        
    mn,mx = ax.get_xlim()
    ax.set_xlim([mn-20,mx+20])
    mn,mx = ax.get_ylim()
    ax.set_ylim([mn+20,mx-20])
    ax.set_xlabel("x (unit)")
    ax.set_ylabel("y (unit)")
    
    s = str(alg).split('.')[1]
    fig.savefig(os.path.join(DIR,s+"_path.png"))
    ax.set_title(s)
       
    dist = wps.dist
    return fig,ax,dist
    
d = data_wp[k]
img = None
for key in d.data:
    img = d.img
    fig,ax,dist = plot(key,d,img,)
    print(f"{key}\n{dist:.4f} units long\n{d.data[key]['time']:.4f}s")
    plt.tight_layout()
    plt.show()

In [ ]:
# PLOT SIMULATION
cutoff = 20*60

def plot(alg,d,img):
    fig, ax = plt.subplots()
    
    ax.imshow(img)
    wps = d.pos
    t = np.array(d.t)
    wps_x = np.array(wps.x)
    wps_y = np.array(wps.y)

    ax.plot(wps_x[np.where(t>=cutoff)],wps_y[np.where(t>=cutoff)],linewidth=3,label='Over 20mins')
    below = ax.plot(wps_x[np.where(t<cutoff)],wps_y[np.where(t<cutoff)],linewidth=3,label='Below 20mins')
    ax.scatter(wps_x[0],wps_y[0],50,color=below[0].get_color(),label=f'Start',zorder=100)

    mn,mx = ax.get_xlim()
    ax.set_xlim([mn-20,mx+20])
    mn,mx = ax.get_ylim()
    ax.set_ylim([mn+20,mx-20])

    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    
    s = str(alg).split('.')[1]
    fig.savefig(os.path.join(DIR,s+"_sim.png"))
    ax.set_title(s)
       
    return fig,ax
    
d = data_sim[k]
for key in d.data:
    alg, output = key
    img = data_wp[k].img
    fig,ax = plot(alg,output,img)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_bar(x,y,x_label,y_label):

    fig,ax = plt.subplots(constrained_layout=True)
    plt.xticks(rotation=45)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.grid(True, linestyle='--', which='major',
                       color='grey', alpha=.25)
    rects = ax.bar(x,y)   

    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

    max_y = np.max(y)
    ax.set_ylim([0, int(np.ceil(max_y / 100.0)) * 110])
    
    return fig,ax

In [ ]:
dists = [item[1]['wps'].dist for item in data_wp[k].data.items()]
times = [item[1]['time'] for item in data_wp[k].data.items()]
flight_times = [np.max(item[1].t) for item in data_sim[k].data]

x = [str(s).split('.')[1] for s in data_wp[k].data.keys()]

fig,ax=plot_bar(x,dists,"","Distance (m)")
fig.savefig(os.path.join(DIR,"unit_distances_all.png"))

fig,ax=plot_bar(x,times,"","Time to calculate (s)")
fig.savefig(os.path.join(DIR,"time_to_calculate_all.png"))

fig,ax=plot_bar(x,flight_times,"","Flight time (s)")
fig.savefig(os.path.join(DIR,"flight_time_all.png"))

In [ ]:
npts = 100
max_cutoff = np.inf
for alg,vehicle in data_sim[k].data:
    file_path = os.path.join(DIR,f"prob_accum_w_t_{str(alg).split('.')[1].lower()}.csv")
    t_arr= []
    c_arr = []
    if os.path.isfile(file_path):
        print(f"File {file_path} found")
        with open(file_path,'r') as f:
            csvreader = csv.DictReader(f) 
            for row in csvreader:
                c_arr.append(float(row['cost']))
                t_arr.append(float(row['time']))
    else:
        wp_arr = []
        t = np.array(vehicle.t)
        x = np.array(vehicle.pos.x)[np.where(t<max_cutoff)]
        y = np.array(vehicle.pos.y)[np.where(t<max_cutoff)]
        t = t[np.where(t<max_cutoff)]
        print(f"{alg} - {npts} points to calculate...")
        for i in range(0,len(x),len(x)//npts) :
            xi = x[i]
            yi = y[i]
            ti = t[i]
            wp_arr.append(Waypoint(xi,yi))
            wps = Waypoints(wp_arr)
            cost = cost_func.calculate(wps,img,R)
            t_arr.append(ti)
            c_arr.append(cost)    
            print(f"{alg} data: {100*i/len(x):.2f}%1 ({ti:.2f}s)")
        with open(file_path,'w') as f:
                csvwriter = csv.writer(f)
                csvwriter.writerow(['time','cost'])
                csvwriter.writerows([(f,g) for f,g in zip(t_arr,c_arr)])
    print(len(c_arr))
    print(f"Final cost for {alg} is {c_arr[-1]:.4f}")
    plt.plot(t_arr,c_arr, label=str(alg).split('.')[1].lower())

plt.xlabel("Time (s)")
plt.ylabel("Accumulated probability")
plt.ylim([0, -1])
plt.legend()
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"probability_over_sim_time.png"))

In [ ]:
total_objects = len(data_sar[k].data)*len(data_sar)
colors = np.array([ImageColor.getcolor(f, "RGB") for f in 
         ['#b71c1c',
          '#311b92',
          '#ff6f00', 
          '#4fc3f7',
          '#d500f9',
          '#1b5e20']])/255

t_hists = {}
for k in data_sim:
    for i,data in enumerate(data_sim[k].data):
        vehicle = data[1]
        alg = data[0]
        v_found = np.array(vehicle.found)[:,0]
        if alg in t_hists:
            t_hists[alg] = np.append(t_hists[alg],v_found)
        else:
            t_hists[alg] = v_found

dct = {}
cutoff = 25*60
fig = plt.figure()

for i,(alg,t_hist) in enumerate(t_hists.items()):

    t_hist = t_hist.astype(float)
    t_hist = t_hist[np.where(t_hist<cutoff)]
    print(f"{alg}: {100*len(t_hist)/total_objects:.2f}% found ({len(t_hist)} out of {total_objects})")
    mean = np.mean(t_hist)

    dct[str(alg).split('.')[1].lower()] = {
        'color':colors[i],
        't_hist_s':t_hist,
        't_hist_min':t_hist/60,
        'mean':np.mean(t_hist),
        'before_endurance_limit':np.sum(t_hist<cutoff)/total_objects
        }
    print(f"{alg} w/ mean time to found = {mean:.2f}s and found before endurance limit ({cutoff}s) = {100*np.sum(t_hist<cutoff)/total_objects:.2f}%")

values = [f['t_hist_min'] for f in dct.values()]
plt.hist(values,bins = 20,label=dct.keys())

plt.ylabel("Found count at time")
plt.xlabel("Time till object found (minutes)")

plt.axvspan(20,30,edgecolor='w',facecolor='r',alpha=0.3,zorder=-100,label='Over endurance',hatch='///')

plt.xlim([0,np.max([np.max(f) for f in values])])
fig.tight_layout()
plt.legend()
plt.show()
fig.savefig(os.path.join(DIR,"time_till_object_found.png"))     

plot_bar([f for f in dct],[100*dct[f]['before_endurance_limit'] for f in dct],'','% found within endurance limit')
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"found_within_endurance.png"))

In [ ]:
x_max = -np.inf
for alg,d in dct.items():
    t_hist = d['t_hist_s']
    y = []
    x = []
    if max(t_hist)>x_max:x_max = max(t_hist)
    for i in np.linspace(0,max(t_hist)):
        x.append(i)
        y.append(100*np.sum(t_hist<i)/total_objects)

    plt.plot(x,y,label=f"{alg}",color=d['color'])

plt.xlim([0,x_max])
plt.ylabel("Percentage found (%)")
plt.xlabel("Endurance limited time-to-find (s)")
fig = plt.gcf()
plt.legend(bbox_to_anchor=(0.0,1))
plt.show()
fig.savefig(os.path.join(DIR,"endurance_limited_time-to-find.png"))
